In [39]:
import pyvisa
import pandas as pd
import numpy as np


In [49]:

rm = pyvisa.ResourceManager()
resources = rm.list_resources()
resource = resources[0]
spectrum_analyzer_resource = rm.open_resource(resource)

In [61]:
spectrum_analyzer_resource.query(':READ:HARMonics:AMPLitude:ALL?')

'-2.042025E+01,3.210000E+02,3.210000E+02\n'

In [50]:
class SpectrumAnalyzer:
    def __init__(self, spectrum_analyzer_resource):
        self.spectrum_analyzer_resource = spectrum_analyzer_resource

    def get_id(self) -> str:
        return self.format_response(self.spectrum_analyzer_resource.query('*IDN?'))
    
    def set_center_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:CENTer {frequency}')
    
    def get_center_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:CENTer?')))
    
    def set_span(self, span : float):
        self.ensure_range(0, 7.5e9, span)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:SPAN {span}')

    def get_span(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:SPAN?')))

    def enable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe ON')

    def disable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe OFF')

    def is_tracking_generator_enable(self)-> bool:
        return self.format_response(self.spectrum_analyzer_resource.query('OUTPut:STATe?')) == '1'
    
    def set_tracking_generator_offset(self, level: int):
        self.ensure_range(-200, 200, level)
        self.spectrum_analyzer_resource.write(f'SOURce:CORRection:OFFSet {level}')

    def get_tracking_generator_offset(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:CORRection:OFFSet?')))
    
    def set_tracking_generator_level(self, level: int):
        self.ensure_range(-40, 0, level)
        self.spectrum_analyzer_resource.write(f'SOURce:POWer:LEVel:IMMediate:AMPLitude {level}')

    def get_tracking_generator_level(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:POWer:LEVel:IMMediate:AMPLitude?')))
    
    def get_power_level_at_current_frequency(self) -> float:
        """
        Queries the power level measured at the current frequency.
        Replace 'MEAS:POW?' with the appropriate SCPI command for your analyzer.
        """
        return float(self.spectrum_analyzer_resource.query('MEAS:POW?'))
    
    def ensure_range(self, min, max, value):
        if value < min and value > max:
            raise ValueError(f'Value must be between {min} and {max}')
    
    def format_response(self, response):
        return response.strip()


In [51]:
spectrum_analyzer = SpectrumAnalyzer(spectrum_analyzer_resource)

In [52]:
spectrum_analyzer.enable_tracking_generator()

In [53]:
spectrum_analyzer.set_center_frequency(1e9)

In [54]:
spectrum_analyzer.set_span(1e9 * 0.1)

In [55]:
spectrum_analyzer.get_power_level_at_current_frequency()

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [11]:
data = [{
    "plant": 'aloe_verra',
    "center_frequency": 0,
    "span": 0,
    "tracking_generator_offset": 0,
    "tracking_generator_level": 0,
}]

In [13]:
df = pd.DataFrame(data)

In [14]:
df

,plant,center_frequency,span,tracking_generator_offset,tracking_generator_level
0,aloe_verra,0,0,0,0
